# Notebook 7: Complete Statistical Analysis Outputs for Professor Yang
## Following Hsu et al. (2018) Methodology - LAGGED Disaster Exposure

## Key Methodological Note
**Per Hsu et al. (2018)**: Disaster exposure is LAGGED by one year.
- We use AFFECTED_RATIO at time t-1 to predict ROA at time t
- This captures the delayed effect of disasters on financial performance
- Formula: ROA_it = β₀ + β₁·AFFECTED_RATIO_i,t-1 + Controls_it + Year_FE + ε_it

## Deliverables
This notebook generates all five deliverables requested:
1. **Complete Analysis Dataset** - All observations with lagged variables
2. **Statistical Model Specification** - Following Hsu et al. (2018)
3. **Descriptive Statistics** - Summary statistics of all variables
4. **Correlation Matrix** - Correlations between all analysis variables
5. **Regression Output Tables** - Full coefficient tables using LAGGED exposure

## Data Sources
- TRI Facility Data (1,148,673 facility-year records)
- SHELDUS Disaster Events (35,283 events, 2009-2023)
- CRSP/Compustat Financial Data
- Final Sample: ~1,787 firm-year observations after lagging (293 firms, 2017-2023)

## Setup: Import Libraries and Mount Drive

In [ ]:
# Mount Google Drive (for Google Colab)
try:
    from google.colab import drive
    drive.mount('/content/drive')
    IN_COLAB = True
except:
    IN_COLAB = False
    print("Not running in Colab, using local paths")

import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("NOTEBOOK 7: STATISTICAL ANALYSIS OUTPUTS FOR PROFESSOR YANG")
print("="*80)
print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

In [ ]:
# Define paths
if IN_COLAB:
    BASE_PATH = Path('/content/drive/MyDrive/Paper1_Dataset')
    PROCESSED_PATH = BASE_PATH / 'processed'
    OUTPUT_DIR = BASE_PATH / 'statistical_outputs_for_professor'
else:
    BASE_PATH = Path('.')
    PROCESSED_PATH = Path('processed')
    OUTPUT_DIR = Path('statistical_outputs_for_professor')

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
print(f"Output directory: {OUTPUT_DIR}")

---
## Part 1: Load and Prepare Complete Analysis Dataset

In [ ]:
print("\n" + "="*80)
print("LOADING DATA")
print("="*80)

# Load facility-level data with disasters
facility_data = pd.read_parquet(PROCESSED_PATH / 'analysis_dataset_complete.parquet')
print(f"\n1. Facility-level data loaded:")
print(f"   Total facility-years: {len(facility_data):,}")
print(f"   With PERMNO: {facility_data['PERMNO'].notna().sum():,}")
print(f"   With disasters: {(facility_data['num_disasters'] > 0).sum():,}")

# Keep only matched facilities
matched = facility_data[facility_data['PERMNO'].notna()].copy()

# Aggregate to company-year level
print(f"\n2. Aggregating to company-year level...")
company_year = matched.groupby(['PERMNO', 'DATA_YEAR']).agg({
    'TRIFD': 'count',  # total facilities
    'num_disasters': 'sum',  # total disasters
    'disaster_exposed': 'sum',  # exposed facilities
    'TICKER': 'first',
}).reset_index()

company_year.columns = ['PERMNO', 'YEAR', 'total_facilities',
                        'num_disasters', 'exposed_facilities', 'TICKER']

# Calculate key variables
company_year['AFFECTED_RATIO'] = company_year['exposed_facilities'] / company_year['total_facilities']
company_year['DISASTER'] = (company_year['num_disasters'] > 0).astype(int)

print(f"   Company-year panel: {len(company_year):,} observations")
print(f"   Unique companies: {company_year['PERMNO'].nunique():,}")

In [ ]:
# Load financial data
print("\n3. Loading Compustat financial data...")
financial_data = pd.read_parquet(PROCESSED_PATH / 'company_year_panel_with_affected_ratio.parquet')

financial_cols = ['PERMNO', 'YEAR', 'TOTAL_ASSETS', 'TOTAL_DEBT', 'NET_INCOME',
                 'TOTAL_REVENUE', 'CASH_FROM_OPS', 'CAPITAL_EXPENDITURE']
financial = financial_data[financial_cols].copy()
print(f"   Financial data: {len(financial):,} company-years")

# Merge disaster exposure with financial data
print("\n4. Merging datasets...")
analysis_data = company_year.merge(financial, on=['PERMNO', 'YEAR'], how='inner')

# Calculate all analysis variables (CONTEMPORANEOUS)
analysis_data['ROA'] = analysis_data['NET_INCOME'] / analysis_data['TOTAL_ASSETS']
analysis_data['LOG_ASSETS'] = np.log(analysis_data['TOTAL_ASSETS'].replace(0, np.nan))
analysis_data['LEVERAGE'] = analysis_data['TOTAL_DEBT'] / analysis_data['TOTAL_ASSETS']
analysis_data['REVENUE_GROWTH'] = analysis_data.groupby('PERMNO')['TOTAL_REVENUE'].pct_change()

print(f"\n   Before lagging:")
print(f"   Total observations: {len(analysis_data):,}")

# ============================================================================
# CRITICAL: CREATE LAGGED VARIABLES (Hsu et al. 2018 Methodology)
# ============================================================================
print("\n" + "="*80)
print("CREATING LAGGED VARIABLES (Hsu et al. 2018 Methodology)")
print("="*80)

# Sort by company and year BEFORE creating lags
analysis_data = analysis_data.sort_values(['PERMNO', 'YEAR']).reset_index(drop=True)

# Create LAGGED disaster exposure variables (t-1)
analysis_data['AFFECTED_RATIO_lag1'] = analysis_data.groupby('PERMNO')['AFFECTED_RATIO'].shift(1)
analysis_data['DISASTER_lag1'] = analysis_data.groupby('PERMNO')['DISASTER'].shift(1)
analysis_data['num_disasters_lag1'] = analysis_data.groupby('PERMNO')['num_disasters'].shift(1)

# Create intensity categories for lagged exposure
def categorize_intensity_lag(ratio):
    if pd.isna(ratio):
        return np.nan
    elif ratio == 0:
        return 'NONE'
    elif ratio <= 0.33:
        return 'LOW'
    elif ratio <= 0.66:
        return 'MEDIUM'
    else:
        return 'HIGH'

analysis_data['INTENSITY_lag1'] = analysis_data['AFFECTED_RATIO_lag1'].apply(categorize_intensity_lag)

# Check lagged variable creation
print(f"\nLagged variable statistics:")
print(f"   AFFECTED_RATIO_lag1 non-null: {analysis_data['AFFECTED_RATIO_lag1'].notna().sum():,}")
print(f"   AFFECTED_RATIO_lag1 mean: {analysis_data['AFFECTED_RATIO_lag1'].mean():.4f}")
print(f"   DISASTER_lag1 mean: {analysis_data['DISASTER_lag1'].mean():.4f}")

# Count observations lost due to lagging (first year per company)
lost_obs = analysis_data['AFFECTED_RATIO_lag1'].isna().sum()
print(f"\n   Observations lost to lagging: {lost_obs:,} (first year per company)")

# Final dataset with lags
print(f"\n   FINAL ANALYSIS DATASET (with lags):")
print(f"   Total observations: {len(analysis_data):,}")
print(f"   With valid lagged exposure: {analysis_data['AFFECTED_RATIO_lag1'].notna().sum():,}")
print(f"   Unique companies: {analysis_data['PERMNO'].nunique():,}")
print(f"   Years: {analysis_data['YEAR'].min()}-{analysis_data['YEAR'].max()}")
print(f"   With complete ROA data: {analysis_data['ROA'].notna().sum():,}")
print("="*80)

---
## DELIVERABLE 1: Complete Analysis Dataset (All Observations)

In [ ]:
print("\n" + "="*80)
print("DELIVERABLE 1: COMPLETE ANALYSIS DATASET")
print("="*80)

# Prepare the complete dataset with all variables
export_columns = [
    'PERMNO',              # Company identifier (CRSP)
    'TICKER',              # Stock ticker symbol
    'YEAR',                # Fiscal year
    'total_facilities',    # Number of TRI facilities
    'exposed_facilities',  # Facilities exposed to disasters
    'num_disasters',       # Total disaster events
    'AFFECTED_RATIO',      # Key independent variable (Hsu et al. 2018)
    'DISASTER',            # Binary disaster indicator
    'ROA',                 # Dependent variable: Return on Assets
    'NET_INCOME',          # Net income ($millions)
    'TOTAL_ASSETS',        # Total assets ($millions)
    'TOTAL_DEBT',          # Total debt ($millions)
    'TOTAL_REVENUE',       # Total revenue ($millions)
    'LOG_ASSETS',          # Control: Log of total assets
    'LEVERAGE',            # Control: Debt/Assets ratio
]

# Only include existing columns
existing_cols = [c for c in export_columns if c in analysis_data.columns]
dataset_export = analysis_data[existing_cols].copy()

# Sort by company and year
dataset_export = dataset_export.sort_values(['PERMNO', 'YEAR'])

# Save to CSV and Excel
csv_file = OUTPUT_DIR / '01_COMPLETE_ANALYSIS_DATASET.csv'
dataset_export.to_csv(csv_file, index=False)
print(f"\n   Saved: {csv_file}")

try:
    xlsx_file = OUTPUT_DIR / '01_COMPLETE_ANALYSIS_DATASET.xlsx'
    dataset_export.to_excel(xlsx_file, index=False, engine='openpyxl')
    print(f"   Saved: {xlsx_file}")
except Exception as e:
    print(f"   Note: Excel export requires openpyxl ({e})")

print(f"\n   Dataset Summary:")
print(f"   - Rows: {len(dataset_export):,}")
print(f"   - Columns: {len(dataset_export.columns)}")
print(f"   - Companies: {dataset_export['PERMNO'].nunique():,}")
print(f"   - Years: {dataset_export['YEAR'].min()}-{dataset_export['YEAR'].max()}")
print(f"\n   Variables included: {list(dataset_export.columns)}")

In [ ]:
# Create data dictionary
print("\n   Creating Data Dictionary...")

variable_descriptions = {
    'PERMNO': 'CRSP permanent company identifier',
    'TICKER': 'Stock ticker symbol',
    'YEAR': 'Fiscal year (2016-2023)',
    'total_facilities': 'Total number of TRI-registered facilities for the company',
    'exposed_facilities': 'Number of facilities in disaster-affected counties',
    'num_disasters': 'Total count of SHELDUS disaster events affecting facilities',
    'AFFECTED_RATIO': 'Proportion of facilities exposed to disasters (0-1)',
    'DISASTER': 'Binary indicator: 1 if any facility exposed to disaster',
    'ROA': 'Return on Assets = Net Income / Total Assets',
    'NET_INCOME': 'Net income in millions USD',
    'TOTAL_ASSETS': 'Total assets in millions USD',
    'TOTAL_DEBT': 'Total debt in millions USD',
    'TOTAL_REVENUE': 'Total revenue in millions USD',
    'LOG_ASSETS': 'Natural logarithm of total assets (size control)',
    'LEVERAGE': 'Financial leverage = Total Debt / Total Assets',
}

data_dict = []
for col in existing_cols:
    non_null = dataset_export[col].notna().sum()
    dtype = str(dataset_export[col].dtype)
    
    if dataset_export[col].dtype in ['float64', 'int64']:
        stats_str = f"Mean={dataset_export[col].mean():.4f}, Std={dataset_export[col].std():.4f}, Min={dataset_export[col].min():.4f}, Max={dataset_export[col].max():.4f}"
    else:
        stats_str = f"{dataset_export[col].nunique()} unique values"
    
    data_dict.append({
        'Variable': col,
        'Description': variable_descriptions.get(col, ''),
        'Type': dtype,
        'Non-Missing': non_null,
        'Statistics': stats_str
    })

data_dict_df = pd.DataFrame(data_dict)
dict_file = OUTPUT_DIR / '01_DATA_DICTIONARY.csv'
data_dict_df.to_csv(dict_file, index=False)
print(f"   Saved: {dict_file}")

print("\n" + data_dict_df.to_string(index=False))

---
## DELIVERABLE 2: Statistical Model Specification

In [ ]:
print("\n" + "="*80)
print("DELIVERABLE 2: STATISTICAL MODEL SPECIFICATION")
print("="*80)

model_specification = """
================================================================================
STATISTICAL MODEL SPECIFICATION
Corporate Resilience to Natural Disasters: Evidence from Manufacturing Firms
Following Hsu et al. (2018) Methodology
================================================================================

RESEARCH QUESTION
-----------------
Do natural disasters affecting a company's facilities impact its financial 
performance, as measured by Return on Assets (ROA)?

================================================================================
CRITICAL METHODOLOGICAL NOTE: LAGGED EXPOSURE (Hsu et al. 2018)
================================================================================

Per Hsu et al. (2018), disaster exposure is LAGGED by one period:
- We use AFFECTED_RATIO at time t-1 to predict ROA at time t
- Rationale: Disasters take time to materially affect financial statements
- The financial impact appears in subsequent reporting periods

================================================================================
VARIABLE DEFINITIONS
================================================================================

DEPENDENT VARIABLE:
-------------------
ROA_t (Return on Assets at time t)
    Formula: ROA = Net Income / Total Assets
    Source: Compustat Annual
    Purpose: Measures firm profitability relative to asset base
    Timing: Contemporaneous (year t)

KEY INDEPENDENT VARIABLE:
-------------------------
AFFECTED_RATIO_t-1 (LAGGED Disaster Exposure Intensity)
    Formula: AFFECTED_RATIO = Exposed Facilities / Total Facilities
    Source: Calculated from TRI facility locations x SHELDUS disaster events
    Purpose: Measures proportion of firm's facilities affected by disasters
    Range: 0 (no exposure) to 1 (all facilities exposed)
    Timing: LAGGED by one year (year t-1)
    Reference: Following Hsu et al. (2018) methodology

CONTROL VARIABLES (Contemporaneous, year t):
--------------------------------------------
1. LOG_ASSETS_t (Firm Size)
    Formula: LOG_ASSETS = ln(Total Assets)
    Timing: Contemporaneous (year t)
    
2. LEVERAGE_t (Financial Structure)
    Formula: LEVERAGE = Total Debt / Total Assets
    Timing: Contemporaneous (year t)

3. YEAR Fixed Effects (μ_t)
    Purpose: Controls for time-varying macroeconomic conditions

================================================================================
REGRESSION MODELS (Hsu et al. 2018 Specification)
================================================================================

MODEL 1: SIMPLE OLS (Baseline)
------------------------------
ROA_it = β₀ + β₁·AFFECTED_RATIO_i,t-1 + ε_it

MODEL 2: WITH FIRM CONTROLS
---------------------------
ROA_it = β₀ + β₁·AFFECTED_RATIO_i,t-1 
            + β₂·LOG_ASSETS_it 
            + β₃·LEVERAGE_it 
            + ε_it

MODEL 3: WITH YEAR FIXED EFFECTS (MAIN SPECIFICATION)
-----------------------------------------------------
ROA_it = β₀ + β₁·AFFECTED_RATIO_i,t-1 
            + β₂·LOG_ASSETS_it 
            + β₃·LEVERAGE_it 
            + Σ(γ_t·YEAR_t)
            + ε_it

Key notation:
    i = firm identifier (PERMNO)
    t = fiscal year
    t-1 = LAGGED (previous year's disaster exposure)
    β₁ = coefficient of interest (disaster impact)
    ε_it = error term

================================================================================
ESTIMATION DETAILS
================================================================================

Estimation Method: Ordinary Least Squares (OLS)
Standard Errors: Robust (heteroskedasticity-consistent)
Software: Python statsmodels

SAMPLE RESTRICTIONS:
1. Manufacturing firms only (SIC codes 20-39)
2. Time period: 2016-2023 (2017-2023 after lagging)
3. Non-missing financial data (ROA, assets, leverage)
4. Non-missing LAGGED disaster exposure (drops first year per firm)

FINAL SAMPLE (after lagging):
- ~1,787 firm-year observations
- 293 unique manufacturing companies
- 7 years (2017-2023, first year lost to lagging)

================================================================================
HYPOTHESIS
================================================================================

H0: β₁ = 0 (Past disasters have no effect on current ROA)
H1: β₁ < 0 (Past disasters negatively impact current ROA)

EXPECTED SIGN: Negative
Rationale:
- Disasters in year t-1 disrupt operations
- Effects materialize in year t financial statements
- Delayed impact on profitability

================================================================================
"""

model_file = OUTPUT_DIR / '02_STATISTICAL_MODEL_SPECIFICATION.txt'
with open(model_file, 'w') as f:
    f.write(model_specification)

print(f"   Saved: {model_file}")
print(model_specification)

---
## DELIVERABLE 3: Descriptive Statistics

In [ ]:
print("\n" + "="*80)
print("DELIVERABLE 3: DESCRIPTIVE STATISTICS")
print("="*80)

# Prepare regression sample (non-missing ROA)
reg_sample = analysis_data[['ROA', 'AFFECTED_RATIO', 'DISASTER', 'LOG_ASSETS', 
                            'LEVERAGE', 'num_disasters', 'total_facilities',
                            'exposed_facilities', 'TOTAL_ASSETS', 'NET_INCOME',
                            'TOTAL_DEBT', 'TOTAL_REVENUE']].dropna(subset=['ROA'])

print(f"\nRegression sample: {len(reg_sample):,} observations\n")

# Calculate comprehensive descriptive statistics
desc_vars = ['ROA', 'AFFECTED_RATIO', 'DISASTER', 'LOG_ASSETS', 'LEVERAGE',
             'num_disasters', 'total_facilities', 'exposed_facilities',
             'TOTAL_ASSETS', 'NET_INCOME', 'TOTAL_DEBT', 'TOTAL_REVENUE']

desc_stats = reg_sample[desc_vars].describe(percentiles=[.01, .05, .25, .50, .75, .95, .99]).T
desc_stats = desc_stats.round(4)

# Add additional statistics
desc_stats['skewness'] = reg_sample[desc_vars].skew().round(4)
desc_stats['kurtosis'] = reg_sample[desc_vars].kurtosis().round(4)

print("DESCRIPTIVE STATISTICS - ALL VARIABLES")
print("-" * 80)
print(desc_stats.to_string())

# Save to CSV and Excel
desc_file_csv = OUTPUT_DIR / '03_DESCRIPTIVE_STATISTICS.csv'
desc_stats.to_csv(desc_file_csv)
print(f"\n   Saved: {desc_file_csv}")

try:
    desc_file_xlsx = OUTPUT_DIR / '03_DESCRIPTIVE_STATISTICS.xlsx'
    desc_stats.to_excel(desc_file_xlsx, engine='openpyxl')
    print(f"   Saved: {desc_file_xlsx}")
except:
    pass

In [ ]:
# Disaster Exposure Distribution
print("\n" + "-"*80)
print("DISASTER EXPOSURE DISTRIBUTION")
print("-"*80)

exposure_bins = [
    ('No exposure (0%)', reg_sample['AFFECTED_RATIO'] == 0),
    ('Low (1-25%)', (reg_sample['AFFECTED_RATIO'] > 0) & (reg_sample['AFFECTED_RATIO'] <= 0.25)),
    ('Medium (26-50%)', (reg_sample['AFFECTED_RATIO'] > 0.25) & (reg_sample['AFFECTED_RATIO'] <= 0.50)),
    ('High (51-75%)', (reg_sample['AFFECTED_RATIO'] > 0.50) & (reg_sample['AFFECTED_RATIO'] <= 0.75)),
    ('Very High (76-100%)', reg_sample['AFFECTED_RATIO'] > 0.75),
]

exposure_data = []
for label, mask in exposure_bins:
    n = mask.sum()
    pct = n / len(reg_sample) * 100
    mean_roa = reg_sample.loc[mask, 'ROA'].mean() if n > 0 else np.nan
    exposure_data.append({
        'Exposure Level': label,
        'N': n,
        'Percentage': round(pct, 1),
        'Mean ROA': round(mean_roa, 4) if not np.isnan(mean_roa) else np.nan
    })

exposure_df = pd.DataFrame(exposure_data)
print(exposure_df.to_string(index=False))

exposure_file = OUTPUT_DIR / '03_EXPOSURE_DISTRIBUTION.csv'
exposure_df.to_csv(exposure_file, index=False)
print(f"\n   Saved: {exposure_file}")

In [ ]:
# Year-by-Year Statistics
print("\n" + "-"*80)
print("YEAR-BY-YEAR STATISTICS")
print("-"*80)

yearly_stats = reg_sample.groupby(analysis_data.loc[reg_sample.index, 'YEAR']).agg({
    'ROA': ['count', 'mean', 'std'],
    'AFFECTED_RATIO': 'mean',
    'DISASTER': 'mean',
    'TOTAL_ASSETS': 'mean'
}).round(4)

yearly_stats.columns = ['N', 'Mean_ROA', 'Std_ROA', 'Mean_Affected_Ratio', 
                        'Disaster_Rate', 'Mean_Assets']
print(yearly_stats.to_string())

yearly_file = OUTPUT_DIR / '03_YEARLY_STATISTICS.csv'
yearly_stats.to_csv(yearly_file)
print(f"\n   Saved: {yearly_file}")

---
## DELIVERABLE 4: Correlation Matrix

In [ ]:
print("\n" + "="*80)
print("DELIVERABLE 4: CORRELATION MATRIX")
print("="*80)

# Variables for correlation matrix
corr_vars = ['ROA', 'AFFECTED_RATIO', 'LOG_ASSETS', 'LEVERAGE', 
             'num_disasters', 'total_facilities', 'exposed_facilities']

# Calculate Pearson correlation matrix
corr_matrix = reg_sample[corr_vars].corr().round(4)

print("\nPEARSON CORRELATION MATRIX")
print("-"*80)
print(corr_matrix.to_string())

# Save correlation matrix
corr_file_csv = OUTPUT_DIR / '04_CORRELATION_MATRIX.csv'
corr_matrix.to_csv(corr_file_csv)
print(f"\n   Saved: {corr_file_csv}")

try:
    corr_file_xlsx = OUTPUT_DIR / '04_CORRELATION_MATRIX.xlsx'
    corr_matrix.to_excel(corr_file_xlsx, engine='openpyxl')
    print(f"   Saved: {corr_file_xlsx}")
except:
    pass

In [ ]:
# Key correlations with significance tests
print("\n" + "-"*80)
print("KEY CORRELATIONS WITH SIGNIFICANCE TESTS")
print("-"*80)

key_pairs = [
    ('ROA', 'AFFECTED_RATIO', 'Main relationship of interest'),
    ('ROA', 'LOG_ASSETS', 'Size-profitability relationship'),
    ('ROA', 'LEVERAGE', 'Leverage-profitability relationship'),
    ('AFFECTED_RATIO', 'LOG_ASSETS', 'Size-exposure relationship'),
    ('AFFECTED_RATIO', 'total_facilities', 'Diversification-exposure'),
]

corr_tests = []
for var1, var2, description in key_pairs:
    r, p = stats.pearsonr(reg_sample[var1].dropna(), 
                          reg_sample.loc[reg_sample[var1].notna(), var2].dropna())
    sig = '***' if p < 0.01 else '**' if p < 0.05 else '*' if p < 0.10 else ''
    corr_tests.append({
        'Variable 1': var1,
        'Variable 2': var2,
        'Correlation': round(r, 4),
        'P-value': round(p, 4),
        'Significance': sig,
        'Description': description
    })

corr_tests_df = pd.DataFrame(corr_tests)
print(corr_tests_df.to_string(index=False))
print("\nSignificance: *** p<0.01, ** p<0.05, * p<0.10")

corr_tests_file = OUTPUT_DIR / '04_KEY_CORRELATIONS.csv'
corr_tests_df.to_csv(corr_tests_file, index=False)
print(f"\n   Saved: {corr_tests_file}")

---
## DELIVERABLE 5: Regression Output Tables (All Coefficients)

In [ ]:
print("\n" + "="*80)
print("DELIVERABLE 5: REGRESSION OUTPUT TABLES")
print("Using LAGGED Disaster Exposure (Hsu et al. 2018)")
print("="*80)

# Prepare regression data - MUST have non-missing LAGGED exposure
reg_data = analysis_data[['ROA', 'AFFECTED_RATIO', 'AFFECTED_RATIO_lag1', 'DISASTER_lag1',
                          'LOG_ASSETS', 'LEVERAGE', 'PERMNO', 'YEAR', 'INTENSITY_lag1']].copy()

# Drop observations with missing lagged exposure (first year per company)
reg_data = reg_data.dropna(subset=['ROA', 'AFFECTED_RATIO_lag1', 'LOG_ASSETS', 'LEVERAGE'])

print(f"\nRegression sample (with LAGGED exposure):")
print(f"   Observations: {len(reg_data):,}")
print(f"   Unique companies: {reg_data['PERMNO'].nunique():,}")
print(f"   Years: {reg_data['YEAR'].min()}-{reg_data['YEAR'].max()}")
print(f"\n   Note: First year per company dropped due to lagging")
print(f"\n   AFFECTED_RATIO_lag1 stats:")
print(f"      Mean: {reg_data['AFFECTED_RATIO_lag1'].mean():.4f}")
print(f"      Std:  {reg_data['AFFECTED_RATIO_lag1'].std():.4f}")
print(f"      Min:  {reg_data['AFFECTED_RATIO_lag1'].min():.4f}")
print(f"      Max:  {reg_data['AFFECTED_RATIO_lag1'].max():.4f}")

In [ ]:
# MODEL 1: Simple OLS with LAGGED exposure
print("\n" + "="*80)
print("MODEL 1: SIMPLE OLS (Hsu et al. 2018)")
print("ROA_t ~ AFFECTED_RATIO_t-1")
print("="*80)

model1 = smf.ols('ROA ~ AFFECTED_RATIO_lag1', data=reg_data).fit()
print(model1.summary())

# Extract coefficients for export
model1_coef = pd.DataFrame({
    'Variable': model1.params.index,
    'Coefficient': model1.params.values.round(6),
    'Std_Error': model1.bse.values.round(6),
    't_statistic': model1.tvalues.values.round(4),
    'P_value': model1.pvalues.values.round(6),
    'CI_Lower_95': model1.conf_int()[0].values.round(6),
    'CI_Upper_95': model1.conf_int()[1].values.round(6)
})

model1_file = OUTPUT_DIR / '05a_MODEL1_SIMPLE_OLS_LAGGED.csv'
model1_coef.to_csv(model1_file, index=False)
print(f"\n   Saved: {model1_file}")

In [ ]:
# MODEL 2: With Controls - LAGGED exposure
print("\n" + "="*80)
print("MODEL 2: WITH FIRM CONTROLS (Hsu et al. 2018)")
print("ROA_t ~ AFFECTED_RATIO_t-1 + LOG_ASSETS_t + LEVERAGE_t")
print("="*80)

model2 = smf.ols('ROA ~ AFFECTED_RATIO_lag1 + LOG_ASSETS + LEVERAGE', data=reg_data).fit()
print(model2.summary())

model2_coef = pd.DataFrame({
    'Variable': model2.params.index,
    'Coefficient': model2.params.values.round(6),
    'Std_Error': model2.bse.values.round(6),
    't_statistic': model2.tvalues.values.round(4),
    'P_value': model2.pvalues.values.round(6),
    'CI_Lower_95': model2.conf_int()[0].values.round(6),
    'CI_Upper_95': model2.conf_int()[1].values.round(6)
})

model2_file = OUTPUT_DIR / '05b_MODEL2_WITH_CONTROLS_LAGGED.csv'
model2_coef.to_csv(model2_file, index=False)
print(f"\n   Saved: {model2_file}")

In [ ]:
# MODEL 3: With Year Fixed Effects - LAGGED exposure (MAIN SPECIFICATION)
print("\n" + "="*80)
print("MODEL 3: WITH YEAR FIXED EFFECTS (Hsu et al. 2018 - MAIN)")
print("ROA_t ~ AFFECTED_RATIO_t-1 + LOG_ASSETS_t + LEVERAGE_t + C(YEAR)")
print("="*80)

model3 = smf.ols('ROA ~ AFFECTED_RATIO_lag1 + LOG_ASSETS + LEVERAGE + C(YEAR)', data=reg_data).fit()
print(model3.summary())

model3_coef = pd.DataFrame({
    'Variable': model3.params.index,
    'Coefficient': model3.params.values.round(6),
    'Std_Error': model3.bse.values.round(6),
    't_statistic': model3.tvalues.values.round(4),
    'P_value': model3.pvalues.values.round(6),
    'CI_Lower_95': model3.conf_int()[0].values.round(6),
    'CI_Upper_95': model3.conf_int()[1].values.round(6)
})

model3_file = OUTPUT_DIR / '05c_MODEL3_YEAR_FE_LAGGED.csv'
model3_coef.to_csv(model3_file, index=False)
print(f"\n   Saved: {model3_file}")

In [ ]:
# SUMMARY TABLE: All Models Compared (LAGGED Exposure)
print("\n" + "="*80)
print("REGRESSION RESULTS SUMMARY (Hsu et al. 2018 - LAGGED Exposure)")
print("="*80)

summary_table = pd.DataFrame({
    'Specification': ['Model 1: Simple OLS', 'Model 2: With Controls', 'Model 3: Year FE (Main)'],
    'AFFECTED_RATIO_lag1_Coef': [model1.params['AFFECTED_RATIO_lag1'], 
                                  model2.params['AFFECTED_RATIO_lag1'],
                                  model3.params['AFFECTED_RATIO_lag1']],
    'AFFECTED_RATIO_lag1_SE': [model1.bse['AFFECTED_RATIO_lag1'],
                                model2.bse['AFFECTED_RATIO_lag1'],
                                model3.bse['AFFECTED_RATIO_lag1']],
    'AFFECTED_RATIO_lag1_Pval': [model1.pvalues['AFFECTED_RATIO_lag1'],
                                  model2.pvalues['AFFECTED_RATIO_lag1'],
                                  model3.pvalues['AFFECTED_RATIO_lag1']],
    'LOG_ASSETS_Coef': [np.nan, model2.params['LOG_ASSETS'], model3.params['LOG_ASSETS']],
    'LEVERAGE_Coef': [np.nan, model2.params['LEVERAGE'], model3.params['LEVERAGE']],
    'R_squared': [model1.rsquared, model2.rsquared, model3.rsquared],
    'Adj_R_squared': [model1.rsquared_adj, model2.rsquared_adj, model3.rsquared_adj],
    'F_statistic': [model1.fvalue, model2.fvalue, model3.fvalue],
    'N': [int(model1.nobs), int(model2.nobs), int(model3.nobs)],
    'Year_FE': ['No', 'No', 'Yes']
}).round(6)

print(summary_table.to_string(index=False))

summary_file = OUTPUT_DIR / '05d_REGRESSION_SUMMARY_LAGGED.csv'
summary_table.to_csv(summary_file, index=False)
print(f"\n   Saved: {summary_file}")

try:
    summary_xlsx = OUTPUT_DIR / '05d_REGRESSION_SUMMARY_LAGGED.xlsx'
    summary_table.to_excel(summary_xlsx, index=False, engine='openpyxl')
    print(f"   Saved: {summary_xlsx}")
except:
    pass

In [ ]:
# Create publication-style regression table (LAGGED)
print("\n" + "="*80)
print("PUBLICATION-STYLE REGRESSION TABLE (Hsu et al. 2018)")
print("="*80)

def format_coef(coef, se, pval):
    """Format coefficient with significance stars"""
    stars = '***' if pval < 0.01 else '**' if pval < 0.05 else '*' if pval < 0.10 else ''
    return f"{coef:.4f}{stars}", f"({se:.4f})"

pub_table = []

# AFFECTED_RATIO_lag1 row (KEY VARIABLE)
row = {'Variable': 'AFFECTED_RATIO (t-1)'}
for i, model in enumerate([model1, model2, model3], 1):
    coef_str, se_str = format_coef(model.params['AFFECTED_RATIO_lag1'], 
                                   model.bse['AFFECTED_RATIO_lag1'],
                                   model.pvalues['AFFECTED_RATIO_lag1'])
    row[f'Model_{i}'] = coef_str
    row[f'Model_{i}_SE'] = se_str
pub_table.append(row)

# LOG_ASSETS row
row = {'Variable': 'LOG_ASSETS (t)'}
row['Model_1'] = ''
row['Model_1_SE'] = ''
for i, model in enumerate([model2, model3], 2):
    coef_str, se_str = format_coef(model.params['LOG_ASSETS'],
                                   model.bse['LOG_ASSETS'],
                                   model.pvalues['LOG_ASSETS'])
    row[f'Model_{i}'] = coef_str
    row[f'Model_{i}_SE'] = se_str
pub_table.append(row)

# LEVERAGE row
row = {'Variable': 'LEVERAGE (t)'}
row['Model_1'] = ''
row['Model_1_SE'] = ''
for i, model in enumerate([model2, model3], 2):
    coef_str, se_str = format_coef(model.params['LEVERAGE'],
                                   model.bse['LEVERAGE'],
                                   model.pvalues['LEVERAGE'])
    row[f'Model_{i}'] = coef_str
    row[f'Model_{i}_SE'] = se_str
pub_table.append(row)

# Intercept row
row = {'Variable': 'Intercept'}
for i, model in enumerate([model1, model2, model3], 1):
    coef_str, se_str = format_coef(model.params['Intercept'],
                                   model.bse['Intercept'],
                                   model.pvalues['Intercept'])
    row[f'Model_{i}'] = coef_str
    row[f'Model_{i}_SE'] = se_str
pub_table.append(row)

# Model statistics
pub_table.append({'Variable': 'Year Fixed Effects', 'Model_1': 'No', 'Model_1_SE': '',
                  'Model_2': 'No', 'Model_2_SE': '', 'Model_3': 'Yes', 'Model_3_SE': ''})
pub_table.append({'Variable': 'R-squared', 
                  'Model_1': f"{model1.rsquared:.4f}", 'Model_1_SE': '',
                  'Model_2': f"{model2.rsquared:.4f}", 'Model_2_SE': '',
                  'Model_3': f"{model3.rsquared:.4f}", 'Model_3_SE': ''})
pub_table.append({'Variable': 'N', 
                  'Model_1': f"{int(model1.nobs):,}", 'Model_1_SE': '',
                  'Model_2': f"{int(model2.nobs):,}", 'Model_2_SE': '',
                  'Model_3': f"{int(model3.nobs):,}", 'Model_3_SE': ''})

pub_df = pd.DataFrame(pub_table)
print(pub_df.to_string(index=False))
print("\nNote: *** p<0.01, ** p<0.05, * p<0.10. Standard errors in parentheses.")
print("      Disaster exposure is LAGGED (t-1) per Hsu et al. (2018)")

pub_file = OUTPUT_DIR / '05e_PUBLICATION_TABLE_LAGGED.csv'
pub_df.to_csv(pub_file, index=False)
print(f"\n   Saved: {pub_file}")

---
## Summary: All Deliverables Generated

In [ ]:
print("\n" + "="*80)
print("GENERATION COMPLETE - ALL DELIVERABLES FOR PROFESSOR YANG")
print("Following Hsu et al. (2018) Methodology with LAGGED Exposure")
print("="*80)

print(f"\nOutput directory: {OUTPUT_DIR}")
print("\nFiles generated:")
print("-" * 80)

for file in sorted(OUTPUT_DIR.glob('*')):
    if file.is_file():
        size_kb = file.stat().st_size / 1024
        print(f"  {file.name:<50} ({size_kb:.1f} KB)")

print("\n" + "="*80)
print("DELIVERABLES SUMMARY")
print("="*80)
print("""
1. COMPLETE ANALYSIS DATASET
   - 01_COMPLETE_ANALYSIS_DATASET.csv/xlsx
   - 01_DATA_DICTIONARY.csv

2. STATISTICAL MODEL SPECIFICATION
   - 02_STATISTICAL_MODEL_SPECIFICATION.txt
   - Following Hsu et al. (2018) LAGGED methodology

3. DESCRIPTIVE STATISTICS
   - 03_DESCRIPTIVE_STATISTICS.csv/xlsx
   - 03_EXPOSURE_DISTRIBUTION.csv
   - 03_YEARLY_STATISTICS.csv

4. CORRELATION MATRIX
   - 04_CORRELATION_MATRIX.csv/xlsx
   - 04_KEY_CORRELATIONS.csv

5. REGRESSION OUTPUT TABLES (LAGGED per Hsu et al. 2018)
   - 05a_MODEL1_SIMPLE_OLS_LAGGED.csv
   - 05b_MODEL2_WITH_CONTROLS_LAGGED.csv
   - 05c_MODEL3_YEAR_FE_LAGGED.csv
   - 05d_REGRESSION_SUMMARY_LAGGED.csv/xlsx
   - 05e_PUBLICATION_TABLE_LAGGED.csv
""")

print("="*80)
print("KEY FINDINGS (Hsu et al. 2018 Methodology - LAGGED Exposure)")
print("="*80)
print(f"""
MAIN RESULT: Effect of LAGGED disaster exposure on ROA

Model 1 (Simple OLS):     beta = {model1.params['AFFECTED_RATIO_lag1']:.4f}, p = {model1.pvalues['AFFECTED_RATIO_lag1']:.3f}
Model 2 (With Controls):  beta = {model2.params['AFFECTED_RATIO_lag1']:.4f}, p = {model2.pvalues['AFFECTED_RATIO_lag1']:.3f}
Model 3 (Year FE - Main): beta = {model3.params['AFFECTED_RATIO_lag1']:.4f}, p = {model3.pvalues['AFFECTED_RATIO_lag1']:.3f}

Sample: {int(model1.nobs):,} firm-year observations (after lagging)
        {reg_data['PERMNO'].nunique()} manufacturing companies
        {reg_data['YEAR'].min()}-{reg_data['YEAR'].max()} period

Model Specification (Hsu et al. 2018):
ROA_it = beta_0 + beta_1 * AFFECTED_RATIO_i,t-1 + beta_2 * LOG_ASSETS_it 
       + beta_3 * LEVERAGE_it + Year_FE + epsilon_it

Key: Disaster exposure is LAGGED by one year (t-1)
     Controls are contemporaneous (year t)
""")
print("="*80)
print("All statistical outputs successfully generated!")
print("="*80)